## Loading the Data

In [1]:
from tensorflow.keras.datasets import imdb

In [2]:
vocab_size = 8000

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

In [4]:
print(type(X_train))
print(type(X_train[5]))
print(X_train[5])

<class 'numpy.ndarray'>
<class 'list'>
[1, 778, 128, 74, 12, 630, 163, 15, 4, 1766, 7982, 1051, 2, 32, 85, 156, 45, 40, 148, 139, 121, 664, 665, 10, 10, 1361, 173, 4, 749, 2, 16, 3804, 8, 4, 226, 65, 12, 43, 127, 24, 2, 10, 10]


In [5]:
maxlen = 200

### Staging and pre-processing our data

In [6]:
from tensorflow.keras import preprocessing

In [7]:
X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [8]:
print(X_train[5])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    1  778  128   74   12  630  163   15    4 1766 7982
 1051    2   32   85  156   45   40  148  139  121  664  665   10   10
 1361  173    4  749    2   16 3804    8    4  226   65   12   43  127
   24 

### Exercise 6.01: Build and train RNN Model for Sentiment Classification

In [9]:
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Flatten, Dense, Embedding, SpatialDropout1D, Dropout

In [11]:
model_rnn = Sequential()

In [ ]:
model_rnn.add(Embedding(vocab_size, output_dim=32))
model_rnn.add(SpatialDropout1D(0.4))

In [12]:
model_rnn.add(SimpleRNN(32))

In [13]:
model_rnn.add(Dropout(0.4))

In [14]:
model_rnn.add(Dense(1, activation='sigmoid'))

In [15]:
model_rnn.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [16]:
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, None, 32)          0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 258,113
Trainable params: 258,113
Non-trainable params: 0
_________________________________________________________________


In [0]:
history_rnn = model_rnn.fit(X_train, y_train, batch_size=128, validation_split=0.2, epochs = 10)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 10s 479us/sample - loss: 0.6924 - acc: 0.5204 - val_loss: 0.6792 - val_acc: 0.5700
Epoch 2/10
20000/20000 [==============================] - 9s 465us/sample - loss: 0.5819 - acc: 0.7024 - val_loss: 0.4309 - val_acc: 0.8258
Epoch 3/10
20000/20000 [==============================] - 9s 451us/sample - loss: 0.4102 - acc: 0.8310 - val_loss: 0.4229 - val_acc: 0.8124
Epoch 4/10
20000/20000 [==============================] - 9s 452us/sample - loss: 0.3330 - acc: 0.8651 - val_loss: 0.3989 - val_acc: 0.8204
Epoch 5/10
20000/20000 [==============================] - 9s 459us/sample - loss: 0.2894 - acc: 0.8865 - val_loss: 0.3850 - val_acc: 0.8290
Epoch 6/10
20000/20000 [==============================] - 9s 453us/sample - loss: 0.2532 - acc: 0.9022 - val_loss: 0.3756 - val_acc: 0.8462
Epoch 7/10
20000/20000 [==============================] - 9s 438us/sample - loss: 0.2240 - acc: 0.9171 - val_l

In [0]:
y_test_pred = model_rnn.predict_classes(X_test)

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
print(accuracy_score(y_test, y_test_pred))

0.85128

### Making Predictions on Unseen Data

In [39]:
inp_review = "An excellent movie!"

In [29]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [30]:
text_to_word_sequence(inp_review)

['an', 'excellent', 'movie']

In [31]:
word_map = imdb.get_word_index()

In [32]:
vocab_map = dict(sorted(word_map.items(), key=lambda x: x[1])[:vocab_size])

In [33]:
def preprocess(review):
    inp_tokens = text_to_word_sequence(review)
    seq = []
    for token in inp_tokens:
        seq.append(vocab_map.get(token))
    return seq

In [40]:
preprocess(inp_review)

[32, 318, 17]

In [41]:
model_rnn.predict_classes([preprocess(inp_review)])

array([[1]])

In [42]:
inp_review = "Don't watch this movie - poor acting, poor script, bad direction."

In [49]:
model_rnn.predict_classes([preprocess(inp_review)])

array([[0]])

### Exercise 6.02: LSTM-Based Sentiment Classification Model

In [0]:
from tensorflow.keras.layers import LSTM

In [0]:
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_size, output_dim=32))
model_lstm.add(SpatialDropout1D(0.4))

In [0]:
model_lstm.add(LSTM(32))

In [0]:
model_lstm.add(Dropout(0.4))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
model_lstm.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, None, 32)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 264,353
Trainable params: 264,353
Non-trainable params: 0
_________________________________________________________________


In [0]:
history_lstm = model_lstm.fit(X_train, y_train, batch_size=128, validation_split=0.2, epochs=5)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 37s 2ms/sample - loss: 0.5569 - acc: 0.7070 - val_loss: 0.3665 - val_acc: 0.8546
Epoch 2/5
20000/20000 [==============================] - 37s 2ms/sample - loss: 0.3336 - acc: 0.8676 - val_loss: 0.3020 - val_acc: 0.8760
Epoch 3/5
20000/20000 [==============================] - 37s 2ms/sample - loss: 0.2695 - acc: 0.8954 - val_loss: 0.2859 - val_acc: 0.8794
Epoch 4/5
20000/20000 [==============================] - 37s 2ms/sample - loss: 0.2417 - acc: 0.9099 - val_loss: 0.3143 - val_acc: 0.8692
Epoch 5/5
20000/20000 [==============================] - 37s 2ms/sample - loss: 0.2173 - acc: 0.9207 - val_loss: 0.2886 - val_acc: 0.8788


In [0]:
y_test_pred = model_lstm.predict_classes(X_test)

In [0]:
print(accuracy_score(y_test, y_test_pred))

0.87032


### Exercise 6.03: GRU-Based Sentiment Classification Model

In [0]:
from tensorflow.keras.layers import GRU

In [0]:
model_gru = Sequential()
model_gru.add(Embedding(vocab_size, output_dim=32))
model_gru.add(SpatialDropout1D(0.4))

In [0]:
model_gru.add(GRU(32, reset_after=False))

In [0]:
model_gru.add(Dropout(0.4))
model_gru.add(Dense(1, activation='sigmoid'))

model_gru.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
model_gru.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          256000    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, None, 32)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 262,273
Trainable params: 262,273
Non-trainable params: 0
_________________________________________________________________


In [0]:
history_gru = model_gru.fit(X_train, y_train, batch_size=128, validation_split=0.2, epochs = 4)

Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 33s 2ms/sample - loss: 0.5554 - acc: 0.7007 - val_loss: 0.4152 - val_acc: 0.8116
Epoch 2/4
20000/20000 [==============================] - 30s 1ms/sample - loss: 0.3372 - acc: 0.8609 - val_loss: 0.3966 - val_acc: 0.8448
Epoch 3/4
20000/20000 [==============================] - 30s 2ms/sample - loss: 0.2766 - acc: 0.8904 - val_loss: 0.3375 - val_acc: 0.8656
Epoch 4/4
20000/20000 [==============================] - 30s 1ms/sample - loss: 0.2478 - acc: 0.9042 - val_loss: 0.2981 - val_acc: 0.8792


In [0]:
y_test_pred = model_gru.predict_classes(X_test)

In [0]:
accuracy_score(y_test, y_test_pred)

0.87156

### Exercise 6.04: Bi-directional LSTM-Based Sentiment Classification Model

In [0]:
from tensorflow.keras.layers import Bidirectional

In [0]:
model_bilstm = Sequential()
model_bilstm.add(Embedding(vocab_size, output_dim=32))
model_bilstm.add(SpatialDropout1D(0.4))

In [0]:
model_bilstm.add(Bidirectional(LSTM(32)))

In [0]:
model_bilstm.add(Dropout(0.4))
model_bilstm.add(Dense(1, activation='sigmoid'))

model_bilstm.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
model_bilstm.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 32)          256000    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, None, 32)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                16640     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 272,705
Trainable params: 272,705
Non-trainable params: 0
_________________________________________________________________


In [0]:
history_bilstm = model_bilstm.fit(X_train, y_train, batch_size=128, validation_split=0.2, epochs = 4)

Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 69s 3ms/sample - loss: 0.5722 - acc: 0.7048 - val_loss: 0.3745 - val_acc: 0.8472
Epoch 2/4
20000/20000 [==============================] - 67s 3ms/sample - loss: 0.3563 - acc: 0.8561 - val_loss: 0.3135 - val_acc: 0.8716
Epoch 3/4
20000/20000 [==============================] - 67s 3ms/sample - loss: 0.2833 - acc: 0.8929 - val_loss: 0.3043 - val_acc: 0.8740
Epoch 4/4
20000/20000 [==============================] - 67s 3ms/sample - loss: 0.2446 - acc: 0.9084 - val_loss: 0.2857 - val_acc: 0.8802


In [0]:
y_test_pred = model_bilstm.predict_classes(X_test)

In [0]:
accuracy_score(y_test, y_test_pred)

0.877

### Exercise 6.05: Stacked LSTM-based Sentiment Classification Model

In [0]:
model_stack = Sequential()
model_stack.add(Embedding(vocab_size, output_dim=32))
model_stack.add(SpatialDropout1D(0.4))

In [0]:
#LSTM Layer 1 - return_sequences is True
model_stack.add(LSTM(32, return_sequences=True))

In [0]:
#LSTM Layer 2 - return_sequences is False
model_stack.add(LSTM(32, return_sequences=False))

In [0]:
model_stack.add(Dropout(0.5))
model_stack.add(Dense(1, activation='sigmoid'))

model_stack.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model_stack.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 32)          256000    
_________________________________________________________________
spatial_dropout1d_9 (Spatial (None, None, 32)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 32)          8320      
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 272,673
Trainable params: 272,673
Non-trainable params: 0
_______________________________________________

In [0]:
history_stack = model_stack.fit(X_train, y_train, batch_size=128, validation_split=0.2, epochs = 4)

Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 76s 4ms/sample - loss: 0.5237 - acc: 0.7286 - val_loss: 0.3558 - val_acc: 0.8552
Epoch 2/4
20000/20000 [==============================] - 72s 4ms/sample - loss: 0.3284 - acc: 0.8687 - val_loss: 0.3360 - val_acc: 0.8702
Epoch 3/4
20000/20000 [==============================] - 72s 4ms/sample - loss: 0.2742 - acc: 0.8944 - val_loss: 0.3436 - val_acc: 0.8666
Epoch 4/4
20000/20000 [==============================] - 72s 4ms/sample - loss: 0.2354 - acc: 0.9096 - val_loss: 0.2827 - val_acc: 0.8826


In [0]:
y_test_pred = model_stack.predict_classes(X_test)

In [0]:
accuracy_score(y_test, y_test_pred)

0.87572